In [1]:
import pandas as pd
import re
import pandas_profiling as pp

In [2]:
df_2021 = pd.read_csv(r"../original_data/Schneider Depression Clinic Database.csv", na_values= ' ')

C:\Users\nogur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df = pd.read_csv('all_columns_2021.csv').rename({'0': 'name'},axis=1)

sciafca_start = df[df['name'] == 'sciafca_timestamp'].index[0]
sciafca_end = df[df['name'] == 'sciafca_complete'].index[0]

In [4]:
columns_dict = {
    'sciafca': df.iloc[sciafca_start+1:sciafca_end]['name'].to_list(),
}

In [5]:
control5_weeks_2021 = df_2021[(df_2021.age_child_pre_first > 0) & (df_2021.age_child_pre_first <= 12) & (df_2021.redcap_event_name=='control_5weeks_arm_1')]
intake_2021 = df_2021[(df_2021.age_child_pre_first > 0) & (df_2021.age_child_pre_first <= 12) & (df_2021.redcap_event_name=='intake_arm_1')]

parameter_5_weeks = control5_weeks_2021[columns_dict['sciafca'] +['age_child_pre_first', 'id', 'record_id', 'redcap_event_name']]
parameter_intake = intake_2021[columns_dict['sciafca']+ ['age_child_pre_first', 'id', 'record_id', 'redcap_event_name']]

### columns

In [7]:
print([i for i in columns_dict['sciafca']])

['sci_af_ca_1', 'sci_af_ca_2', 'sci_af_ca_3', 'sci_af_ca_4', 'sci_af_ca_5', 'sci_af_ca_6', 'sci_af_ca_7', 'sci_af_ca_8', 'sci_af_ca_9', 'sci_af_ca_10', 'sci_af_ca_11', 'sci_af_ca_12', 'sci_af_ca_13', 'sci_af_ca_14', 'sci_af_ca_15', 'sci_af_ca_16', 'sci_af_ca_17', 'sci_af_ca_18', 'sci_af_ca_19', 'sci_af_ca_20', 'sci_af_ca_21', 'sci_af_ca_22', 'sci_af_ca_23', 'sci_af_ca_24', 'sci_af_ca_25', 'sci_af_ca_26', 'sci_af_ca_27', 'sci_af_ca_28', 'sci_af_ca_29', 'sci_af_ca_30', 'sci_af_ca_31', 'sci_af_ca_32', 'sci_af_ca_33', 'sci_af_ca_34', 'sci_af_ca_35', 'sci_af_ca_36', 'sci_af_ca_37', 'sci_af_ca_38', 'sci_af_ca_39', 'sci_af_ca_40']


## intake

In [9]:
intake_full_size = parameter_intake.shape[0]
print('full size =', parameter_intake.shape)

all_questions_intake = parameter_intake.dropna(subset=columns_dict['sciafca'], how='any')
all_questions_size = parameter_intake.dropna(subset=columns_dict['sciafca'], how='any').shape[0]
print('#N participants that filled all the questions =', all_questions_size, f'\n{(all_questions_size/intake_full_size)*100}%')

any_questions_intake = parameter_intake.dropna(subset=columns_dict['sciafca'], how='all')
any_questions_size = parameter_intake.dropna(subset=columns_dict['sciafca'], how='all').shape[0]
print('\n#N participants that filled all the questions =', any_questions_size, f'\n{(any_questions_size/intake_full_size)*100}%')


full size = (86, 44)
#N participants that filled all the questions = 25 
29.069767441860467%

#N participants that filled all the questions = 44 
51.162790697674424%


## control 5 weeks

In [10]:
five_weeks_full_size = parameter_5_weeks.shape[0]
print('full size =', parameter_5_weeks.shape)

all_questions_5_weeks = parameter_5_weeks.dropna(subset=columns_dict['sciafca'], how='any')
all_questions_size = parameter_5_weeks.dropna(subset=columns_dict['sciafca'], how='any').shape[0]
print('#N participants that filled all the questions =', all_questions_size, f'\n{(all_questions_size/five_weeks_full_size)*100}%')

any_questions_5_weeks = parameter_5_weeks.dropna(subset=columns_dict['sciafca'], how='all')
any_questions_size = parameter_5_weeks.dropna(subset=columns_dict['sciafca'], how='all').shape[0]
print('\n#N participants that filled all the questions =', any_questions_size, f'\n{(any_questions_size/five_weeks_full_size)*100}%')


full size = (35, 44)
#N participants that filled all the questions = 18 
51.42857142857142%

#N participants that filled all the questions = 21 
60.0%


## both events

In [11]:
ids_in_both_times = parameter_intake[parameter_intake['id'].isin(parameter_5_weeks['id'])]['id']
print('#N subjects in both events', ids_in_both_times.shape[0])



ids_in_both_dicts_all_q = all_questions_intake[all_questions_intake['id'].isin(all_questions_5_weeks['id'])]['id']
print('#N subjects that filled in both events, all questions', ids_in_both_dicts_all_q.shape[0])



ids_in_both_dicts = any_questions_intake[any_questions_intake['id'].isin(any_questions_5_weeks['id'])]['id']

print('#N subjects that filled in both events', ids_in_both_dicts.shape[0])


merged = pd.merge(any_questions_intake, any_questions_5_weeks, on='id', how='outer', suffixes=('_intake', '_5_weeks'))
print('#N subjects total', merged.shape[0])


#N subjects in both events 35
#N subjects that filled in both events, all questions 13
#N subjects that filled in both events 16
#N subjects total 49


In [12]:
to_profile = merged
profile = pp.ProfileReport(to_profile, title="sci_af_ca_2021")
profile.to_file("sci_af_ca_2021.html")



Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]